# 배경 이미지 생성

여기서는 모델 학습에 사용할 배경 이미지를 생성합니다.

아스팔트나 콘크리트 바닥같은 아무 사진이나 구해오면 되지만 그건 귀찮고,

더 편한 방법이 있으므로 그냥 대충 만듭니다.

목표는 400장인데 약간 더 많이 만들고 눈으로 골라내겠습니다.


## 방법

1. 바운딩 박스가 2개 이상이면 그냥 버립니다.
2. 바운딩 박스의 왼/오른쪽은 너비, 위/아래쪽은 높이가 됩니다.
3. 일단은 둘 모두 320 이상이 될 수 있는 이미지가 충분히 있는지를 보려고 합니다.
4. 그게 장수가 충분히 많다면 거기서 고르고, 너무 적으면 기준을 좀 낮춥니다.
5. json에서 읽어도 되지만 yolov5 라벨도 만들었으므로 그걸로 하는 게 좋을 것 같습니다.



근데 생각해보니 기왕 이렇게 할거면 train에서 배경을 뽑아오는 것 보다 학습 동안 어차피
안 쓸 validation 셋에서 뽑아보는 것도 좋을 것 같습니다.

숫자만 맞으면 됩니다.


In [11]:
import os
import random

# from collections import namedtuple
from pathlib import Path
from pprint import pprint
from typing import (
    Any, NamedTuple, NewType, Optional, Sequence, Tuple, Union
)

from PIL import Image


random.seed()



In [36]:
Rect = NewType('Rect', Tuple[int, int, int, int])

class Label(object):
    '''yolov5 라벨 한 줄(=바운딩 박스 1개)을 읽어 해석합니다.'''
    classid: int
    centerx: int
    centery: int
    width: int
    height: int
    dim: int

    def __init__(self, text: str, dim: int = 640) -> None:
        self.dim = int(dim)

        cid, cx, cy, w, h = map(float, text.strip().split(' '))
        self.classid = cid
        self.centerx = round(cx * self.dim)
        self.centery = round(cy * self.dim)
        self.width = round(w * self.dim)
        self.height = round(h * self.dim)

    def biggest_rect(self, threshold: float) -> Optional[Rect]:
        '''바운딩 박스를 제외한 가장 큰 크랍 렉트를 반환합니다.
        
        가장 큰 렉트가 ``threshold``보다 작으면 ``None``을 반환합니다.
        ``threshold``는 픽셀 값이 아닌 비율입니다.
        '''
        threshold *= self.dim
        edges = [
            self.centerx - self.width//2,
            self.dim - self.centerx - self.width//2,
            self.centery - self.height//2,
            self.dim - self.centery - self.height//2,
        ]
        limiting_edge = max(edges)
        if limiting_edge < threshold:
            return None

        edge_index = edges.index(limiting_edge)
        rect: Rect
        if edge_index == 0:
            # left vertical
            rect = (0, 0, limiting_edge, limiting_edge)
        elif edge_index == 1:
            # right vertical
            rect = (
                self.dim - 1 - limiting_edge,
                0,
                self.dim - 1,
                limiting_edge,
            )
        elif edge_index == 2:
            # upper horizontal
            rect = (0, 0, limiting_edge, limiting_edge)
        else:
            # lower horizontal
            rect = (
                0,
                self.dim - 1 - limiting_edge,
                limiting_edge,
                self.dim - 1,
            )

        return rect


In [19]:
def nominate(top: Path, min_dim: float = 0.5) -> Sequence[Tuple[Path, Rect]]:
    # cache = set()
    top = Path(top)
    nominees = []
    for pathname in top.rglob('*.txt'):
        with open(pathname, 'r', encoding='utf-8') as txt_in:
            lines = txt_in.readlines()
            if len(lines) > 1:
                # 2줄 이상인 파일은 곧바로 무시합니다.
                continue

            label = Label(lines[0])
            rect = label.biggest_rect(threshold=min_dim)
            if rect is not None:
                nominees.append(
                    (pathname.relative_to(top).with_suffix('.jpg'), rect)
                )

    return nominees


In [37]:
nominees = nominate('./dataset/validation-labels/')
len(nominees)

6354

validation만 쳐도 6354개가 살아남은 것 같습니다.


In [38]:
pprint(random.sample(nominees, 5))

[(PosixPath('형광등/콤팩트형/25_X001_C015_1203/25_X001_C015_1203_1.jpg'),
  (0, 301, 338, 639)),
 (PosixPath('형광등/콤팩트형/25_X030_C038_1209/25_X030_C038_1209_0.jpg'),
  (318, 0, 639, 321)),
 (PosixPath('비닐/포장제/15_X009_C706_1218/15_X009_C706_1218_4.jpg'),
  (0, 240, 399, 639)),
 (PosixPath('플라스틱류/장남감/24_X529_C999_0328/24_X529_C999_0328_0.jpg'),
  (0, 255, 384, 639)),
 (PosixPath('나무/주걱/13_X238_C024_0226/13_X238_C024_0226_0.jpg'),
  (0, 317, 322, 639))]


좀 아닌 것 같은 배경 이미지는 눈으로 보고 지워도 400개는 남기기 위해 800개를 뽑겠습니다.


In [39]:
chosen = random.sample(nominees, 800)

골라낸 이미지를 크랍해서 저장합니다.

ssh 환경에서 이걸 확인하기는 좀 불편하므로 로컬로 옮겨서 작업합니다.


In [40]:
def write_images(
    targets: Sequence[Tuple[Path, Rect]], src: Path, dst: Path
) -> Sequence[Path]:
    fail = []
    src = Path(src)
    dst = Path(dst)
    for path, rect in targets:
        src_ = src / path
        try:
            with Image.open(src_) as image_in:
                # image_out = image_in.crop((*rect[0], *rect[1]))
                image_out = image_in.crop(rect)
                dst_ = dst / path.name
                image_out.save(dst_)
        except Exception as why:
            print(why)
            fail.append(path)

    return fail


In [41]:
Path('./dataset/training/bg').mkdir(parents=True, exist_ok=True)
fail = write_images(chosen, './dataset/validation', './dataset/training/bg')
len(fail)


0